<a href="https://colab.research.google.com/github/AryanilAD/Corporate-Training-Mini_Projects/blob/main/NN_on_pima_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this mini-project, I implemented a neural network model for binary classification using Keras and Scikit-learn. I started by preprocessing the dataset, splitting it into training and testing sets, and scaling the features. I then used GridSearchCV to tune hyperparameters such as learning rate and dropout rate. After finding the best hyperparameters, I created a final model and trained it on the training set. Finally, I evaluated the model on both the training and testing sets to measure its performance. This project helped me understand the importance of hyperparameter tuning and how it can improve the performance of a neural network model.

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

# Load the dataset
df = pd.read_csv("/content/diabetes.csv")

In [23]:
# Split into input (X) and output (Y) variables
X = df.drop('Outcome', axis=1)
Y = df['Outcome']

In [24]:
# Split the dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [25]:
# Standardize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
# Define the base model
def create_model(optimizer='adam', activation='relu', kernel_initializer='glorot_uniform', neurons=1, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(neurons, input_dim=8, kernel_initializer=kernel_initializer, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=kernel_initializer, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [27]:
# Create classifier model
classifier = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-27-13fe41dfede8>:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=create_model, verbose=0)


In [28]:
# Define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Perform grid search for batch size and epochs
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

# Summarize results for batch size and epochs
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.765471 using {'batch_size': 20, 'epochs': 100}


In [29]:
from keras.optimizers import Adam

# Define the grid search parameters
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)

# Create a function to build the model with the specified learning rate
def create_model(learning_rate):
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=10, verbose=0)

# Perform grid search for learning rate
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

# Summarize results for learning rate
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


<ipython-input-29-808df8d1303c>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=10, verbose=0)


Best: 0.747553 using {'learning_rate': 0.001}


In [30]:
# Define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
kernel_initializer = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(activation=activation, kernel_initializer=kernel_initializer)

# Perform grid search for activation function and kernel initializer
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

# Summarize results for activation function and kernel initializer
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.656377 using {'activation': 'softsign', 'kernel_initializer': 'normal'}


In [31]:
# Define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)

# Perform grid search for number of neurons in activation layer
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, Y_train)

# Summarize results for number of neurons in activation layer
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.617161 using {'neurons': 20}


In [32]:
def create_model(optimizer='adam', activation='relu', kernel_initializer='glorot_uniform', neurons=1, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_scaled.shape[1], kernel_initializer=kernel_initializer, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the final model with optimum hyperparameters
best_params = grid_result.best_params_

# Extract the best hyperparameters or use default values if not found
optimizer = best_params.get('optimizer', 'adam')
activation = best_params.get('activation', 'relu')
kernel_initializer = best_params.get('kernel_initializer', 'glorot_uniform')
neurons = best_params.get('neurons', 1)
dropout_rate = best_params.get('dropout_rate', 0.0)

final_model = create_model(optimizer=optimizer, activation=activation,
                            kernel_initializer=kernel_initializer, neurons=neurons,
                            dropout_rate=dropout_rate)

# Train the final model with optimum hyperparameters
final_model.fit(X_train_scaled, Y_train, epochs=25, batch_size=10)

# Evaluate the final model
Y_pred = final_model.predict(X_test_scaled)
accuracy = accuracy_score(Y_test, Y_pred.round())
print("Test Accuracy: %.2f%%" % (accuracy * 100))

# Evaluate the final model on training data
train_loss, train_accuracy = final_model.evaluate(X_train_scaled, Y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (train_accuracy * 100))

Epoch 1/28
62/62 [==============================] - 1s 2ms/step - loss: 0.5580 - accuracy: 0.7443
Epoch 2/28
62/62 [==============================] - 0s 2ms/step - loss: 0.5230 - accuracy: 0.7638
Epoch 3/28
62/62 [==============================] - 0s 2ms/step - loss: 0.5003 - accuracy: 0.7655
Epoch 4/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4862 - accuracy: 0.7720
Epoch 5/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.7785
Epoch 6/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4687 - accuracy: 0.7834
Epoch 7/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4634 - accuracy: 0.7801
Epoch 8/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4593 - accuracy: 0.7834
Epoch 9/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4559 - accuracy: 0.7915
Epoch 10/28
62/62 [==============================] - 0s 2ms/step - loss: 0.4531 - accuracy: 0.7866
Epoch 11/28
62/62 [